<a href="https://colab.research.google.com/github/gallileoG/2015/blob/master/bcgov_rs_2023_2a.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Rasters
###### bcgov-rs-workshop-2023

In this notebook, we will look at ways to load, analyze, and display rasters using Python.

## Setup

We'll start by installing a few additional modules to our environment that we'll use later. Custom environments do not persist between Google Colab sessions, so you will need to install these dependencies each time you start/restart the runtime.

In [ ]:
!pip install rasterio

Import all the necessary dependencies.

In [ ]:
import folium
import numpy as np
import matplotlib.pyplot as plt
import requests

from PIL import Image, ImageFilter

import rasterio
from rasterio.warp import calculate_default_transform, reproject, Resampling

## Nonspatial Rasters (numpy)

Rasters are sometimes called "grids" for good reason: they are literally grids of values (usually integers or floating values [decimals]). The standard Python data structures for gridded numeric data are [numpy arrays](https://numpy.org/doc/stable/reference/arrays.html).

Numpy arrays have a defined `shape`, consisting of sizes of each dimension. For example, a 2D array has two dimensions: `[rows, columns]` (note the order). Here, we define a `[2, 2]` numpy array from a list of lists of numbers:

In [ ]:
vals = [
    [1, 2],
    [3, 4]
]

arr = np.array(vals)
arr

We can also instantiate a random numpy array of any shape:

In [ ]:
arr = np.random.randint(low=0, high=256, size=[3,4])
print(arr)

Other useful array generation methods are:
- [np.ones()](https://numpy.org/doc/stable/reference/generated/numpy.ones.html)
- [np.zeros()](https://numpy.org/doc/stable/reference/generated/numpy.zeros.html)
- [np.arange()](https://numpy.org/doc/stable/reference/generated/numpy.arange.html)
  - note that arange will create an array on a single row, so it is often followed by [reshape()](https://numpy.org/doc/stable/reference/generated/numpy.reshape.html#)

In [ ]:
arr_ones = np.ones([2, 2])
arr_zeros = np.zeros([2, 2])
arr_arange = np.arange(9).reshape([3, 3])

print(f"arr_ones: \n{arr_ones}")
print(f"arr_zeros: \n{arr_zeros}")
print(f"arr_arange: \n{arr_arange}")

In addition to holding gridded numeric values, numpy arrays have properties:
- `shape`: a 2D array (a flat table) has a shape like `[rows, columns]` (note the order), where each index of row/column holds a pixel value. We will also see 3D arrays, which often either have a shape like `[row, column, band]` or `[band, row, column]`, depending on the needs of the module we intend to use. Be aware that numpy arrays can have more dimensions, as well.
- `dtype`: this is the data type of the values in the arrays. Each array has only one data type. Data type corresponds to pixel bit depth. Numpy has numerous built-in [data types](https://numpy.org/doc/stable/reference/arrays.scalars.html#sized-aliases). We will most often encounter `np.uint8` (unsigned 8-bit integers [0 - 255]).
- `size`: the number of elements in an array
- `ndim`: the number of dimensions in an array
- `nbytes`: the number of bytes of the array
- numerous summary statistics are available, as well, such as `np.mean()` shown below

In [ ]:
print(arr)
print(f"Shape: {arr.shape}")
print(f"Data Type: {arr.dtype}")
print(f"Size: {arr.size}")
print(f"Number of Dimensions: {arr.ndim}")
print(f"Array Mean: {np.mean(arr)}")
print(f"Number of Bytes: {arr.nbytes}")

Notice that our array was created as dtype = int64 (signed 64-bit) by default, and occupies 96 bytes. 1 byte = 8 bits. 64 bits = 8 bytes. There are twelve 8-byte values in our array.

When we created the array, we specified that we wanted integer values in the range 0 - 255, which can fit into the `np.uint8` data type. We can change the dtype using the `astype` method. Notice how much space we save simply by choosing the minimum necessary data type. Saving a few bytes here and there is not very important in this context, but at scale can be a huge savings.

In [ ]:
arr8 = arr.astype(np.uint8)
print(arr8)
print(f"Shape: {arr8.shape}")
print(f"Data Type: {arr8.dtype}")
print(f"Size: {arr8.size}")
print(f"Number of Dimensions: {arr8.ndim}")
print(f"Array Mean: {np.mean(arr8)}")
print(f"Number of Bytes: {arr8.nbytes}")

We can display 2D arrays using matplotlib. Notice that values close to 0 are black, and those close to 255 are white. Try changing the `size` parameter to create more pixels in the array (e.g. `size=[300, 400]`). Also notice that we can control the dtype during array creation.

In [ ]:
arr = np.random.randint(low=0, high=256, size=[3,4], dtype=np.uint8)
print(arr, arr.dtype)
plt.imshow(arr, cmap="gray")

We can also create multiband rasters with numpy. These rasters take the shape of `[rows, columns, bands]`. In the example below, we create an array with the shape `[5, 4, 3]`, resulting in an image with 5 rows, 4 columns, and 3 bands. 3-band rasters are common in remote sensing data, because they are easily displayed as color images by assigning one band each to correspond to the colors red, green, and blue ("RGB" images). Matplotlib automatically displays 3-band images as RGB. Notice that pixels correspond to RGB values (e.g. an array value of `[255, 0, 0]` would result in a pixel that appears very red). Try entering some of the RGB values into an [RGB Calculator](https://www.w3schools.com/colors/colors_rgb.asp) to convince yourself that numpy and matplotlib are working together to display our numeric values as colored pixels.

In [ ]:
arr = np.random.randint(low=0, high=256, size=[5,4,3], dtype=np.uint8)
print(arr, arr.dtype)
plt.imshow(arr)

We can reference pixel values by indices (recall that Python indices are 0-based):

- `arr[0, 2, 1]`: reference the value in the first row, third column, and second band

We can also reference "slices" of data, using [slice notation](https://numpy.org/doc/stable/user/basics.indexing.html#slicing-and-striding).

- `arr[0, 0, :]`: first row, first column, all band values
- `arr[:2, :2, :]`: first two rows, first two columns, all band values
- `arr[1:3, :3, :]`: rows 2-3, first three columns, all band values 

Try referencing specific pixel values and slices of pixels.

In [ ]:
selected_pixels = arr[1:3, :3, :]
plt.imshow(selected_pixels)

## Nonspatial Rasters (pillow)

So far, we've generated image data ourselves, but usually we will read a file or retrieve image data from an API.

The main module for reading nonspatial imagery is [pillow](https://pillow.readthedocs.io/en/stable/) (derived from "PIL": the Python Imaging Library). We can instantiate an `Image` object with `Image.open()` - we can open image files (e.g. jpeg, png, etc.) or load raw image data from an API like below. Below, we retrieve an image from [cataas](https://cataas.com/#/) ("Cat As A Service").

Note that the URL ends with a specific ID, meaning that we will retrieve a specific image. You can omit the ID to retrieve a random cat image, however, there is no guarantee that all images will work nicely in later cells (i.e. they may have a different number of dimensions).

In [ ]:
url = "https://cataas.com/cat/g9ECV9Q1e9nciaoY"

im = Image.open(requests.get(url, stream=True).raw)
im

We can load the PIL image object to a numpy array:

In [ ]:
arr = np.array(im)
arr

And print out common numpy array metadata:

In [ ]:
print(f"Shape: {arr.shape}")
print(f"Data Type: {arr.dtype}")
print(f"Size: {arr.size}")
print(f"Number of Dimensions: {arr.ndim}")
print(f"Array Mean: {np.mean(arr)}")
print(f"Number of Bytes: {arr.nbytes}")

And extract and slice pixels as we have already seen. Here, we show each band separately. Notice that "red" pixels are relatively high (white) in the red band, and similar for other colours. Note this method for displaying multiple outputs.

In [ ]:
plt.imshow(arr[:, :, 0], cmap="gray")
plt.title('Red Channel')
plt.show()
plt.imshow(arr[:, :, 1], cmap="gray")
plt.title('Green Channel')
plt.show()
plt.imshow(arr[:, :, 2], cmap="gray")
plt.title('Blue Channel')
plt.show()

`pillow` has lots of image processing functionality that we won't cover in this workshop, but we will take a quick look at [ImageFilters](https://pillow.readthedocs.io/en/stable/reference/ImageFilter.html).

There are several built-in image filters that we can see below. You can chain the output of one filter into another filter. For example, you could first blur an image and then extract visual contours for a rudimentary object detection algorithm.

Below, we resize the original image just to limit the size of the output. It's not strictly necessary.

Note another method for displaying multiple outputs.

In [ ]:
scale_factor = 0.5
small_im = im.resize(
    (
      int(im.width * scale_factor), int(im.height * scale_factor)
    )
  )

im1 = small_im.filter(ImageFilter.BLUR)
im2 = small_im.filter(ImageFilter.CONTOUR)
im3 = small_im.filter(ImageFilter.DETAIL)
im4 = small_im.filter(ImageFilter.EDGE_ENHANCE)
im5 = small_im.filter(ImageFilter.EDGE_ENHANCE_MORE)
im6 = small_im.filter(ImageFilter.EMBOSS)
im7 = small_im.filter(ImageFilter.FIND_EDGES)
im8 = small_im.filter(ImageFilter.SHARPEN)
im9 = small_im.filter(ImageFilter.SMOOTH)
im10 = small_im.filter(ImageFilter.SMOOTH_MORE)

display(
    im1,
    im2,
    im3,
    im4,
    im5,
    im6,
    im7,
    im8,
    im9,
    im10
)

## Spatial Rasters (rasterio)

The previous sections have shown us how to create and load nonspatial images, but most remote sensing images are spatial. We want to make measurements and show them on maps!

Let's start by collecting a spatial image. The link below is from [BC Map & Orthos](https://a100.gov.bc.ca/ext/mtec/public/products/mapsheet). You can use the provided link, or find your own by:
- navigate to [BC Map & Orthos](https://a100.gov.bc.ca/ext/mtec/public/products/mapsheet)
- select a mapsheet on the map (make note of the mapsheet name)
- on the `NTS 250K` tab, under the `Raster Topographic Maps (NTS)` section, right-click and copy the link for the `BC Albers TIFF` download button.

The download function will save the topographic map delivery zipfile to this Google Colab session data. Check the `Files` (folder icon) on the lefthand side of the screen. You will likely have `refresh` the view, as well.

In [ ]:
def download_url(url, save_path, chunk_size=128):
    r = requests.get(url, stream=True)
    with open(save_path, 'wb') as fd:
        for chunk in r.iter_content(chunk_size=chunk_size):
            fd.write(chunk)

url = "https://pub.data.gov.bc.ca/datasets/177864/tif/bcalb/093n/bc_093n_xc10m_bcalb.zip"
zip_path = "bcalb.zip"
download_url(url, zip_path)

[Rasterio](https://rasterio.readthedocs.io/en/latest/) is the most common Python module for spatial raster data. In addition to handling imagery in a similar way to how we have already seen `numpy` and `pillow` do it, `rasterio` handles the spatial aspect of our data.

If you've downloaded your own image above, be sure to update the `mapsheet` variable so the contained tiff file can be found.

We load imagery with rasterio using `rasterio.open()`. We usually point it directly to a `.tif` (or `.tiff`) file or url, but in this case we have a local zipfile containing a `.tif`. Luckily, there is [support for that](https://rasterio.readthedocs.io/en/latest/topics/datasets.html#dataset-identifiers). The main thing to note is the syntax: `'zip:///path/to/file.zip!/folder/file.tif'`

Once opened, pay attention to the `profile`. This is where all of the dataset's spatial metadata is found. Note that there is information about the data type, image dimensions, number of bands ("count"), and coordinate reference system ("crs"). Also, note that we retrieved this metadata very quickly - we have not read any pixel values yet, only retrieved metadata about the dataset as a whole.


In [ ]:
mapsheet = "093N"
tif_path = f"zip://{zip_path}!/{mapsheet}.tif"

dataset = rasterio.open(tif_path)
dataset.profile

We can read actual pixel values with the `read()` method. Note that band indexing starts at 1 - you will retrieve an error if you attempt `dataset.read(0)`. Also, observe that the result of reading pixel values is our good friend, numpy array!

In [ ]:
band1 = dataset.read(1)
band1

And now you can display it, or use in any other way that you have learned to use numpy arrays.

In [ ]:
plt.imshow(band1)

We can read individual bands and combine into a multiband numpy array with [np.dstack()](https://numpy.org/doc/stable/reference/generated/numpy.dstack.html). Note that the array shape correspond to the dataset profile values: `[height, width, count]`

In [ ]:
print(f"Profile: {dataset.profile}")
r = dataset.read(1)
g = dataset.read(2)
b = dataset.read(3)
band_stack = np.dstack([r, g, b])
print(f"Array Shape: {band_stack.shape}")

plt.imshow(band_stack)

And we can slice our multiband array as we have seen before:

In [ ]:
plt.imshow(band_stack[500:1500, 3500:4500, :])

Our next goal will be to display this image correctly geolocated on a `folium` map. We will create a Raster Layer ImageOverlay (https://python-visualization.github.io/folium/modules.html#module-folium.raster_layers), which requires the image array, plus bounding box coordinates *in latitude/longitude* (EPSG:4326). You may have noticed our image was provided in BC Albers (EPSG:3005):

In [ ]:
print(f"Profile: {dataset.profile}")
print(f"CRS: {dataset.crs}")

If we print the bounds of the original image, the coordinates will be in BC Albers:

In [ ]:
print(f"BBox: {dataset.bounds}")

In order to get lat/long coordinates, we must* reproject our original image and use the bounds from the reprojected image. Rasterio provides a method for [reprojecting images](https://rasterio.readthedocs.io/en/latest/topics/reproject.html). We will save a new file called `reprojected.tif` in the session storage.

*in some cases, it may also work to reproject only the corner coordinates, but for this demonstration we will reproject the entire image.

In [ ]:
dst_crs = 'EPSG:4326'

transform, width, height = calculate_default_transform(
    dataset.crs, dst_crs, dataset.width, dataset.height, *dataset.bounds)
kwargs = dataset.meta.copy()
kwargs.update({
    'crs': dst_crs,
    'transform': transform,
    'width': width,
    'height': height
})

with rasterio.open('reprojected.tif', 'w', **kwargs) as dst:
    for i in range(1, dataset.count + 1):
        reproject(
            source=rasterio.band(dataset, i),
            destination=rasterio.band(dst, i),
            src_transform=dataset.transform,
            src_crs=dataset.crs,
            dst_transform=transform,
            dst_crs=dst_crs,
            resampling=Resampling.nearest)

Read the new file into a rasterio dataset object:

In [ ]:
reprojected_dataset = rasterio.open('reprojected.tif')
reprojected_dataset.profile

We can read each data band into a numpy array, and combine the individual bands into an RGB composite using np.dstack().

In [ ]:
r = reprojected_dataset.read(1)
g = reprojected_dataset.read(2)
b = reprojected_dataset.read(3)
band_stack = np.dstack([r, g, b])
band_stack.shape

plt.imshow(band_stack)

And also notice that the dataset bounds are reported in coordinates of lat/long:

In [ ]:
reprojected_dataset.bounds

Finally, we can create a folium map, assemble the bounds coordinates, and create an ImageOverlay:

In [ ]:
m = folium.Map(location=[54.361, -124.885], zoom_start=5)

bounds = [
    [reprojected_dataset.bounds.bottom, reprojected_dataset.bounds.left],
    [reprojected_dataset.bounds.top, reprojected_dataset.bounds.right]
]
folium.raster_layers.ImageOverlay(band_stack, bounds, pixelated=False).add_to(m)

folium.LayerControl().add_to(m)
m

## Extra Credit Ideas

- explore [LidarBC](https://www2.gov.bc.ca/gov/content/data/geographic-data-services/lidarbc)
  - download a DEM tif (not laz file) and upload to this notebook
  - open the image using rasterio
  - inspect the metadata
  - display the image
  - try to display values only above or below certain values
  - try extracting vector features (lines) delineating values at a certain elevation (see [here](https://rasterio.readthedocs.io/en/latest/topics/features.html))
  - display the DEM on a folium map
  - if you extracted features above, try to display them on a folium map
- look for an image (.tif) in the [Canada SPOT Orthoimages STAC Catalog](https://stacindex.org/catalogs/spot-orthoimages-canada-2005#/). We will discuss STAC more in the next session, but you can click links to drill deeper into the catalog until you find images under the "Assets" tab in the associated item.
  - download the image and upload to this notebook
  - read with raterio
  - manipulate in numpy as you please
  - display on a folium map
- explore [numpy](https://numpy.org/) (this could be a whole workshop)
  - poke around in the [reference](https://numpy.org/doc/stable/reference/index.html#reference) just to see how many functions are available
  - [X-ray image processing tutorial](https://numpy.org/numpy-tutorials/content/tutorial-x-ray-image-processing.html)
- explore [pillow](https://pillow.readthedocs.io/en/stable/)
  - [tutorial](https://pillow.readthedocs.io/en/stable/handbook/tutorial.html)
- there is another common nonspatial image module that we will not cover in this workshop: [OpenCV](https://docs.opencv.org/4.x/). OpenCV is similar to pillow, but with a computer vision (CV) focus. Google Colab notebooks are preloaded with OpenCV.
  - [OpenCV Python tutorials](https://docs.opencv.org/4.x/d6/d00/tutorial_py_root.html)
  - I find the OpenCV documentation/tutorials quite difficult to parse, so often turn to external resources, like:
    - [PyImageSearch](https://pyimagesearch.com/start-here/)
    - [LearnOpenCV](https://learnopencv.com/getting-started-with-opencv/)
- 